## Vamos iniciar nosso notebook chamando todas as bibliotecas que usaremos. Nessa etapa, nada precisa ser modificado.

In [ ]:
!pip install arrow bokeh paho.mqtt -q
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import pprint
import numpy as np
import arrow
import requests
import json
import pandas as pd
from threading import Timer

#Url de publicacao dos dados
pub_url = 'https://data.demo.konkerlabs.net/pub/'
#Url da API
base_api = 'https://api.demo.konkerlabs.net'
#Application padrão
application = 'default'

In [ ]:
# Credenciais do usuário da Plataforma
username = ''
password = ''

In [ ]:
device_name = "Cloud"

## Usando a API da Konker para obter os dados e analisa-los localmente
Para iniciar esse trabalho, vamos primeiro conectar na API da Konker. A API usa OAuth2, então primeiro vamos obter as credenciais.

In [ ]:
client = BackendApplicationClient(client_id=username)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url='{}/v1/oauth/token'.format(base_api),
                                       client_id=username,
                                       client_secret=password)

In [ ]:
devices = oauth.get("https://api.demo.konkerlabs.net/v1/{}/devices/".format(application)).json()['result']
for dev in devices:
    print(dev)

Vamos procurar pelo dispositivo Cloud na sua lista de dispositivos:

In [ ]:
guid_dev=""
for dev in devices:
    if dev['name'] == device_name:
        guid_dev = dev['guid']

print("O GUID do dispositivo é: "+ guid_dev)

Caso você consiga ver o GUID do dispositivo, significa que está tudo funcionando bem. Caso o GUID não apareça, revise o nome do dispositivo no Notebook e o nome escolhido na plataforma para garantir que eles possuem a mesma grafia.

In [ ]:
dt_start = arrow.utcnow().to('America/Sao_Paulo').floor('day')
dt_start = dt_start.shift(days=-20)
stats = oauth.get("https://api.demo.konkerlabs.net/v1/{}/incomingEvents?q=device:{} timestamp:>{}&sort=oldest&limit=10000".format(application,guid_dev,dt_start.isoformat())).json()['result']
print(stats)

In [ ]:
stats_df = pd.json_normalize(stats).set_index('timestamp')
stats_df

Agora você pode trabalhar com os dados no formato tabular, separando os dados de cada experimento separadamente.Cada dado está em um canal:

Experimento 1: "temperature"<br />
Experimento 2: "mnist"<br />
Experimento 3: "dogcat"

In [ ]:
temperature = stats_df[stats_df['incoming.channel']=='temperature']
mnist = stats_df[stats_df['incoming.channel']=='mnist']
dogcat = stats_df[stats_df['incoming.channel']=='dogcat']

Vamos tomar como exemplo a primeira aplicação, pois as variáveis são basicamente as mesmas em todas as demais. Na primeira aplicação temos as variáveis:

**_ts**: timestamp de chegada do dado na plataforma de IoT; </br >
**edge_ts**: timestamp da saída do dado do servidor de borda; </br >
**cloud_latency**: tempo de execução da inferência na nuvem (ms); </br >
**edge_latency**: tempo de execução da inferência na borda (ms);</br >
**dev_latency**:tempo de execução no dispositivo (ms) - Essa variável só existe nessa aplicação, pois o KMeans pode ser executado no dispostivivo.

A ideia desse laboratório é calcular onde é mais benéfico rodar a aplicação de Aprendizado de Máquina testada, supondo que a inferência deve ser enviada de volta para o dispositivo (logo deve-se somar o tempo borda->nuvem e nuvem->borda que vamos assumir serem idênticos). Suponha também que a latência entre o dispositivo e a borda seja desprezível. Dessa forma, para que seja benéfico rodar na nuvem, a condição seria:

cloud_latency+2$*$(_ts - edge_ts - cloud_latency) < edge_latency </br >
cloud_latency+2$*$(_ts - edge_ts - cloud_latency) < dev_latency

Da mesma forma, para que seja benéfico rodar na borda, temos a condição:

edge_latency < cloud_latency+2$*$(_ts - edge_ts - cloud_latency) </br >
edge_latency < dev_latency



In [ ]:
#Nesse exemplo, estou usando apenas os últimos 10 pontos, por questão de estabilidade do Google Colab (antes disso a latência era ordens de magnitude maior). Recomendo que vocês testem essa variável.

app1_cloud = (2*(temperature['payload._ts'][-10:] - temperature['payload.edge_ts'][-10:]) - temperature['payload.cloud_latency'][-10:]).mean()
app1_edge = (temperature['payload.edge_latency'][-10:]).mean()
app1_dev = (temperature['payload.dev_latency'][-10:]).mean()

In [ ]:
print('Tempo total para inferência na Nuvem: ' + str(app1_cloud)+ ' ms')
print('Tempo total para inferência na Borda: ' + str(app1_edge)+ ' ms')
print('Tempo total para inferência no Dispotivo: ' + str(app1_dev)+ ' ms')

Após calcular qual a melhor camada para rodar cada uma das três aplicações de Aprendizado de Máquina, descreva em um relatório sua resposta e se esse era o resultado que você esperaria. Discuta se as soluções encontradas fazem sentido para você. Adicione no relatório também uma explicação a respeito de como foi feita a montagem do experimento, ou seja, qual o equipamento utilizado, como foi conectado e o código que você utilizou.